# Uses the rasters representing the maximum flood height to analyse the extents of inundation due to the simualted SLR scenarios

1. Define the path of the simulation results (stored locally)

2. Define the number of SLR cases used in the simulations to read each results raster and map unique values to the combined raster.

3. Compare each of the rasters and combine the rasters to show increasing flood inundation due to increasing SLR

4. Finally, mask the ocean from the combined raster and save the combined raster as a geotiff with ditinctive values for each of the SLR cases

In [30]:
import numpy as np
import subprocess
import fiona
import rasterio
from rasterio.mask import mask
import shapely
import shapely.geometry
from shapely.ops import transform

import ogr
import osr
from pyproj import Proj, transform
import geopandas as gpd
import pandas as pd
import gdal
import struct
import matplotlib.pyplot as plt
import os
os.environ["PROJ_LIB"] = "C:/ProgramData/Anaconda3/Library/share"
from mpl_toolkits.basemap import Basemap

from pyproj import Proj, transform
import matplotlib

In [31]:
##############################################################################
## Run through defined ensembles to compare flooded areas for the ensembles ##
## Compare different SLR scenarios in this example ###########################
##############################################################################

## Define case no.
case = "designCase06"

## Define ensembles to be read from the max height rasters and reverse the array
ensembles = np.arange(1,7)
ensembles = ensembles[::-1]

# Create the base file path based on the case number
FilePath = "W:/work/FloodModelling/Thailand_2020/SwiftProjectFiles/"+case+"/"

# Define an example raster, to create the combined raster
InRasterName = "results_grid_height_max.0005.tif"
InRasterPath = FilePath + InRasterName
ds_temp = gdal.Open(InRasterPath)
xsize = ds_temp.RasterXSize
ysize = ds_temp.RasterYSize
#inRasterValues = ds_temp.GetRasterBand(1).ReadAsArray()

## Name to save the combined raster
outRasterCombined = case+"_"+"CombinedSLR.tif"

# load geotiff driver 
driver_tiff = gdal.GetDriverByName('GTiff')

# Create new raster and set the crs
ds = driver_tiff.Create(outRasterCombined, xsize = xsize, \
                    ysize = ysize, bands =1,\
                    eType = gdal.GDT_Float32)
ds.SetGeoTransform(ds_temp.GetGeoTransform())
ds.SetProjection(ds_temp.GetProjection())

# initialize output raster band 
combinedBand = np.empty([ysize,xsize])  

for i in ensembles:
    InRasterName = "results_grid_height_max.000"+str(i)+".tif"
    InRasterPath = FilePath + InRasterName
    ds_temp = gdal.Open(InRasterPath)
    inRasterValues = ds_temp.GetRasterBand(1).ReadAsArray()
    
    for y in range(ysize):
        for x in range(xsize):
            if (inRasterValues[y][x] > 0.1):
                combinedBand[y][x] = int(i)
                
            else:
                pass
            
# Write the values to saved raster
ds.GetRasterBand(1).WriteArray(combinedBand)   

# close raster file
ds = None
print('saved: ' + outRasterCombined)            


saved: designCase06_CombinedSLR.tif


In [37]:
##############################
## Masking the saved raster ##
##############################

maskShp = 'Mask/MaskProjectedUpdated.shp'
historicalDataShp = 'rd1_20111030_rad_55_0058_0060_Projected.shp'
maskedRaster = case+"_"+"CombinedSLR_Masked" + ".tif"

# read in the mask shapefile
with fiona.open(maskShp, "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]
    
inRasterPath = outRasterCombined

with rasterio.open(inRasterPath) as src:
    # mask the raster with the shapefile geometry
    out_image, out_transform = mask(src, geoms, invert=True)
    out_meta = src.meta.copy()
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

#save the masked raster
with rasterio.open(maskedRaster, "w", **out_meta) as dest:
    dest.write(out_image)
